In [1]:
import json
from schema import MaterialBase, MaterialLayer, Construction, ConstructionSet, Zone, Template, Library, UmiBase

# Schema

Defining all objects with Pydantic classes and Field constructors let's us trivially construct a JSON Schema.  This schema can be saved to file, passed around, etc.  It can be used with `datamodel-codegen` to succesfully rebuild the Pydantic classes, however you would lose any rich functions that go along with the objects, e.g. a function which computes an assemblies R-Value.  

In [2]:
schema = Library.model_json_schema(mode="serialization")

print(json.dumps(schema, indent=4))


{
    "$defs": {
        "Construction": {
            "description": "A Construction is a set of Material Layers that represent an assembly.",
            "properties": {
                "id": {
                    "format": "uuid4",
                    "title": "Object ID",
                    "type": "string"
                },
                "Layers": {
                    "description": "The list of Material Layers which define this construction",
                    "items": {
                        "$ref": "#/$defs/MaterialLayer"
                    },
                    "title": "Material Layers",
                    "type": "array"
                }
            },
            "required": [
                "id",
                "Layers"
            ],
            "title": "Construction",
            "type": "object"
        },
        "ConstructionSet": {
            "description": "A Zone Construction Set defines the assemblies for the walls, roofs, etc that make up\nthe en

# Test


We can define a library at will similarly to before.  Here, we define a few `Material` objects as well as a `MaterialLayer` which uses one of the materials.

In [3]:
generic_mat_a = MaterialBase(
    Conductivity=1,
    SpecificHeat=800,
)

generic_mat_b = MaterialBase(
    Conductivity=2,
    SpecificHeat=1000,
)

shared_face = MaterialLayer(
    Thickness=0.1,
    Material=generic_mat_a,
)

shared_face.Material.Conductivity = 2

# Demonstrate that nested objects work as expected
assert generic_mat_a.Conductivity == 2

Next, we can make a `Construction` which makes use of the shared face, as well as defines a new `MaterialLayer` inline using the other `Material`.

In [4]:
wall = Construction(
    Layers=[
        shared_face,
        MaterialLayer(Thickness=0.05, Material=generic_mat_b),
        shared_face,
    ]
)

# Demontsrate that the shared face is really shared
assert wall.Layers[0] is wall.Layers[2]

Now let's define a `Zone`, using our existing `wall` for most of the `Construction` objects within the `ConstructionSet` (defined inline), and a separate inline definition for the `Construction` used for the `Slab`.  This new `Construction` will use a mix of existing `MaterialLayer` objects, new layers with existing materials, and new layers with new materials.

In [5]:
perim = Zone(
    DaylightWorkplaneHeight=1,
    Constructions=ConstructionSet(
        Partition=wall,
        Roof=wall,
        Ground=wall,
        Wall=wall,
        Slab=Construction(
            Layers=[
                shared_face,
                shared_face,
                shared_face,
                MaterialLayer(
                    Thickness=0.5,
                    Material=generic_mat_b,
                ),
                MaterialLayer(
                    Thickness=0.5,
                    Material=MaterialBase(
                        Conductivity=0.35,
                        SpecificHeat=500,
                    ),
                ),
            ]
        ),
    ),
)

Now let's make a copy of the zone for the core.

In [6]:
core = perim.model_copy()

core.DaylightWorkplaneHeight = perim.DaylightWorkplaneHeight + 1

# demonstrate that ids change when copying
assert core.id != perim.id

# Demonstrate that top level fields are decoupled
assert core.DaylightWorkplaneHeight != perim.DaylightWorkplaneHeight

# Demonstrate the nested fields are still shared
assert core.Constructions is perim.Constructions

Finally, let's make our `Library`.

In [7]:
lib = Library(
    Templates=[
        Template(
            Perimeter=perim,
            Core=perim.model_copy(deep=True),
            YearFrom=2000,
            YearTo=2001,
            Country="USA",
        )
    ]
)

# Serializing

Saving a library is as easy as dumping it.  

In [8]:
data = lib.model_dump_json(indent=4)

print(data)

{
    "id": "aa6c9cd3-9028-4efd-98fc-97b147b4523f",
    "Templates": [
        {
            "id": "4300c8e3-8733-4f97-a64f-35bcfc54825c",
            "Perimeter": {
                "id": "d05a79f9-5081-4ddc-8e9e-b8076836903d",
                "DaylightWorkplaneHeight": 1.0,
                "Constructions": {
                    "id": "abb80437-1731-420e-a892-3ad83731770c",
                    "Partition": {
                        "id": "84c99200-d918-47e1-9922-7bc253dbb8da",
                        "Layers": [
                            {
                                "id": "b61072cc-9a91-4d7b-806f-b7fe7e153f39",
                                "Thickness": 0.1,
                                "Material": {
                                    "id": "1b49424a-c629-49b8-b6b7-077e4718f1cb",
                                    "Conductivity": 2.0,
                                    "SpecificHeat": 800.0
                                }
                            },
                

Notice that this is shown as a deeply nested object, however all objects have `id`s.  Normally, if we tried to deserialize this object using Pydantic, we would lose all shared referencing...

However, there is a neat/simple trick in the validation logic when objects are craeted which fetches an existing version of the object from a cache if the id is already represented in the cache.

We can see that cache:

In [9]:
list(UmiBase.all.values())

[MaterialBase(id=UUID('1b49424a-c629-49b8-b6b7-077e4718f1cb'), Conductivity=2, SpecificHeat=800.0),
 MaterialBase(id=UUID('3c1d3de9-15e4-4954-bb5f-c2571451e675'), Conductivity=2.0, SpecificHeat=1000.0),
 MaterialLayer(id=UUID('b61072cc-9a91-4d7b-806f-b7fe7e153f39'), Thickness=0.1, Material=MaterialBase(id=UUID('1b49424a-c629-49b8-b6b7-077e4718f1cb'), Conductivity=2, SpecificHeat=800.0)),
 MaterialLayer(id=UUID('ec638dd3-38a5-4f81-8273-601b8e3d7a8d'), Thickness=0.05, Material=MaterialBase(id=UUID('3c1d3de9-15e4-4954-bb5f-c2571451e675'), Conductivity=2.0, SpecificHeat=1000.0)),
 Construction(id=UUID('84c99200-d918-47e1-9922-7bc253dbb8da'), Layers=[MaterialLayer(id=UUID('b61072cc-9a91-4d7b-806f-b7fe7e153f39'), Thickness=0.1, Material=MaterialBase(id=UUID('1b49424a-c629-49b8-b6b7-077e4718f1cb'), Conductivity=2, SpecificHeat=800.0)), MaterialLayer(id=UUID('ec638dd3-38a5-4f81-8273-601b8e3d7a8d'), Thickness=0.05, Material=MaterialBase(id=UUID('3c1d3de9-15e4-4954-bb5f-c2571451e675'), Conductiv

However, we can also serialize our data as a flat list of objects by type, equivalent to how they would be stored in a relational database.

In [10]:
flat_serialization = UmiBase.flat_serialization(as_json=True)
print(flat_serialization)

{
    "MaterialBase": [
        {
            "id": "1b49424a-c629-49b8-b6b7-077e4718f1cb",
            "Conductivity": 2,
            "SpecificHeat": 800.0
        },
        {
            "id": "3c1d3de9-15e4-4954-bb5f-c2571451e675",
            "Conductivity": 2.0,
            "SpecificHeat": 1000.0
        },
        {
            "id": "8c1e74ab-d7c6-4d6c-a730-bcf61336585f",
            "Conductivity": 0.35,
            "SpecificHeat": 500.0
        }
    ],
    "MaterialLayer": [
        {
            "id": "b61072cc-9a91-4d7b-806f-b7fe7e153f39",
            "Thickness": 0.1,
            "Material": {
                "id": "1b49424a-c629-49b8-b6b7-077e4718f1cb"
            }
        },
        {
            "id": "ec638dd3-38a5-4f81-8273-601b8e3d7a8d",
            "Thickness": 0.05,
            "Material": {
                "id": "3c1d3de9-15e4-4954-bb5f-c2571451e675"
            }
        },
        {
            "id": "d7a80f14-9373-42c5-97dd-dc6eedd44f27",
            "Thickne

Reconstructing from the flat, effectively tabular data is a little trickier to do automatically without resorting to a database.